In [8]:
import sys
import os
from dotenv import load_dotenv
from pathlib import Path

In [10]:
# Load secret .env file
load_dotenv(Path("home/chibu/projects/future_modern/code/aeria/.env"))

False

In [11]:
key = os.getenv('SC_API_KEY')

In [12]:
if key is not None:
    print("Key loaded.")
else:
    print("The key did not load")

The key did not load


In [5]:
import networkx as nx
from functools import partial
from utils import get_results, handle_http_errors

In [ ]:
import soundcloud

In [6]:
def getAttr(resource, attr):
    #getattr with error handling
    if hasattr(resource, attr): return getattr(resource, attr)
    return None

#fix name for specialized getAttr functions
getUsername = partial(getAttr, attr='username')
getid = partial(getAttr, attr='id')
getUserid = partial(getAttr, attr='user_id')

In [7]:
def profile2id(profile):
    #convert a profile to an id
    print("get user id result is {}".format(getUserid(profile)))
    print("get id result is {}".format(getid(profile)))
    return getUserid(profile) if isinstance(getUserid(profile), int) else getid(profile)

In [8]:
def profile2username(profile):
    #convert a profile to a username
    return getUsername(profile) if isinstance(getUsername(profile), str) else str(getid(profile))

In [9]:
# A global profile graph used to iterate through the various algorithms.
# Each node is profile id, with edges weighted by activity between then.
profile_graph = nx.MultiDiGraph()


In [10]:

client = soundcloud.Client(client_id='454aeaee30d3533d6d8f448556b50f23')
name = input("Enter a soundcloud artist to analyze: ")

Enter a soundcloud artist to analyze: Sybyr


In [11]:
# Artist of interest

search = client.get('/users/', q = name)[0]

HTTPError: 401 Client Error: Unauthorized for url: https://api.soundcloud.com/users?q=Sybyr&client_id=454aeaee30d3533d6d8f448556b50f23

In [9]:
search.__dict__

{'obj': {'avatar_url': 'https://i1.sndcdn.com/avatars-jDucfE5tYobXzMC1-o76g6g-large.jpg',
  'id': 13377746,
  'kind': 'user',
  'permalink_url': 'https://soundcloud.com/sybyr',
  'uri': 'https://api.soundcloud.com/users/13377746',
  'username': 'Sybyr (ProdByLandfill)',
  'permalink': 'sybyr',
  'last_modified': '2020/11/22 13:23:27 +0000',
  'first_name': 'ProdbyLandfill.com',
  'last_name': '',
  'full_name': 'ProdbyLandfill.com',
  'city': '',
  'description': 'bookings/email: sybyrcentral@gmail.com\nAnti-World: http://antiworldwide.co (@antiworldglobe) on all networks\n(Prod. by Landfill) at http://prodbylandfill.com',
  'country': 'United States',
  'track_count': 442,
  'public_favorites_count': 0,
  'reposts_count': 110,
  'followers_count': 53041,
  'followings_count': 24,
  'plan': 'Pro Unlimited',
  'myspace_name': None,
  'discogs_name': None,
  'website_title': 'click Here',
  'website': 'http://antiworld.lnk.to/SYBYR',
  'comments_count': 110,
  'online': False,
  'likes_c

In [10]:
print("Artist interpreted as: %s" % getUsername(search))

Artist interpreted as: Sybyr (ProdByLandfill)


In [11]:
uid2username_cache = {}
tid2username_cache = {}
cid2username_cache = {}
tid2userid_cache = {}
cid2userid_cache = {}

# Creating Decorator: handle_http_errors

In [12]:
@handle_http_errors
def userid2username(uid):
    global uid2username_dict
    username = uid2username_cache.get(uid, None)
    if username is not None: 
        return username
    # username is none, we don't have it in cache
    
    uid_str = str(uid)
    get = '/users/{}'.format(uid_str)
    result = client.get(get)
    username = result.username
    uid2username_cache[uid] = username
    return username

@handle_http_errors
def trackid2username(tid):
    global tid2username_dict
    username = tid2username_cache.get(uid, None)
    if username is not None: 
        return username
    # username is none, we don't have it in cache
    
    tid_str = str(tid)
    get = '/tracks/{}'.format(tid_str)
    result = client.get(get)
    username = result.user['username']
    tid2username_cache[tid] = username
    return username

@handle_http_errors
def commentid2username(cid):
    global cid2username_dict
    username = cid2username_cache.get(uid, None)
    if username is not None: 
        return username
    # username is none, we don't have it in cache
    
    cid_str = str(cid)
    get = '/comments/{}'.format(cid_str)
    result = client.get(get)
    username = result.user['username']
    cid2username_cache[cid] = username
    return username

@handle_http_errors
def trackid2userid(tid):
    global tid2userid_dict
    userid = tid2userid_cache.get(tid, None)
    if userid is not None: 
        return userid
    # username is none, we don't have it in cache
    
    tid_str = str(tid)
    get = '/tracks/{}'.format(tid_str)
    result = client.get(get)
    userid = result.user['id']
    tid2userid_cache[tid] = userid
    return userid

@handle_http_errors
def commentid2userid(cid):
    global cid2userid_dict
    username = cid2username_cache.get(cid, None)
    if username is not None: 
        return username
    # username is none, we don't have it in cache
    
    cid_str = str(cid)
    get = '/comments/{}'.format(cid_str)
    result = client.get(get)
    userid = result.user['id']
    cid2userid_cache[cid] = userid
    return userid
    

# get social data functions

In [13]:
def getRelationships(profile, client, url): 
    return get_results(client, url)

@handle_http_errors
def getFollows(profile):
    # get all profiles a user follows
    follows = get_results(client, '/users/{0:s}/followings/'.format(str(profile)))
    return follows

@handle_http_errors
def getFollowers(profile):
    # get all profiles that follow a user
    followers = get_results(client, '/users/{0:s}/followers/'.format(str(profile)))
    return followers

@handle_http_errors
def getUserFavorites(profile):
    user_favorites = get_results(client, '/users/{0:s}/favorites/'.format(str(profile)))
    return user_favorites

@handle_http_errors
def getUserComments(profile):
    comments = get_results(client, '/users/{0:s}/comments/'.format(str(profile)))
    return comments

@handle_http_errors
def getTracks(profile):
    tracks = get_results(client, '/users/{0:s}/tracks/'.format(str(profile)))
    return tracks

@handle_http_errors
def getTrackComments(track):
    track_comments = get_results(client, '/tracks/{0:s}/comments/'.format(str(track)))
    return track_comments

@handle_http_errors
def getTrackFavoriters(track):
    track_favoriters = get_results(client, '/tracks/{0:s}/favoriters/'.format(str(track)))
    return track_favoriters

# build graph functions

In [14]:
def addProfileNx(profile, profileGraph):
    profileGraph.add_node(profile)
    #return profileGraph

def addPairNx(profile, neighbor, profileGraph):
    addProfileNx(profile)
    addProfileNx(neighbor)
    
def addFollowNx(profile, follow, profileGraph):
    # enforce that already existing follows don't get added again
    if not profileGraph.has_edge(profile, follow, 'soundcloud_follow'):
        profileGraph.add_edge(profile, follow, 'soundcloud_follow',weight=1)

def addFollowsNx(profile, follows, profileGraph):
    for follow in follows:
        addFollowNx(profile, follow, profileGraph)

def addFollowersNx(profile, followers, profileGraph):
    for follower in followers:
        addFollowNx(follower, profile, profileGraph)

# enforce that the same favorite isn't counted twice
# keep a list of tracks as an edge attribute
def addFavoriteNx(profile, favorite, track, profileGraph):
    # if we already saw one profile favorite the other before...
    if profileGraph.has_edge(profile, favorite, 'soundcloud_favorite'):
        # but not for this track
        if track not in profileGraph.edges[profile, favorite, 'soundcloud_favorite']['tracks']:
            profileGraph.edges[profile, favorite, 'soundcloud_favorite']['tracks'] = profileGraph.edges[profile, favorite, 'soundcloud_favorite'].get('tracks', []) + [track]
    else:
        # we never saw this favorite from this profile before, so create a new edge "type" or key
        profileGraph.add_edge(profile, favorite, 'soundcloud_favorite')#,weight=1)
        profileGraph.edges[profile, favorite, 'soundcloud_favorite']['tracks'] = [track]
    profileGraph.edges[profile, favorite,'soundcloud_favorite']['weight'] = len(profileGraph.edges[profile, favorite, 'soundcloud_favorite']['tracks'])        


def addUserFavoritesNx(profile, user_favorites, profileGraph):
    for track, user_favorite in user_favorites:
        addFavoriteNx(profile, user_favorite, track, profileGraph)
            
def addTrackFavoriters(profile, track_favoriters, profileGraph):
    for track, track_favoriter in track_favoriters:
        addFavoriteNx(track_favoriter, profile, track, profileGraph)

# enforce that the same comment isn't counted twice
# keep a dictionary of track-comments pairs as an edge attribute

def addCommentNx(profile, commented_profile, track, comment, profileGraph):
    # if we already saw one profile comment the other before...
    if profileGraph.has_edge(profile, commented_profile, 'soundcloud_comment'):
        # and that profile already commented on this track
        if track in profileGraph.edges[profile, commented_profile, 'soundcloud_comment']['tracks_comments'].keys():
            # but we never saw this comment from this profile for this track
            if comment not in profileGraph.edges[profile, commented_profile, 'soundcloud_comment']['tracks_comments'][track]:
                # then add this comment from this user to this track
                profileGraph.edges[profile, commented_profile, 'soundcloud_comment']['tracks_comments'][track] = profileGraph.edges[profile, favorite, 'soundcloud_comment']['tracks_comments'].get(track, []) + [comment]
        else:
            # we've seen a comment from this profile to the other, but never on this track before, so we must add the new track to the tracks_comments edge attribute
            profileGraph.edges[profile, commented_profile, 'soundcloud_comment']['tracks_comments'][track] = [comment]
    else:
        # we've never seen a comment from this profile to the other, so create a new edge "type" or key for soundcloud comments
        profileGraph.add_edge(profile, commented_profile, 'soundcloud_comment')
        profileGraph.edges[profile, commented_profile, 'soundcloud_comment']['tracks_comments'] = {track: [comment]} 
    profileGraph.edges[profile, commented_profile,'soundcloud_comment']['weight'] = profileGraph.edges[profile, favorite, 'soundcloud_comment']['tracks_comments']
    
def addUserCommentsNx(profile, user_comments, profileGraph):
    for track, comment, user_comment in user_comments:
        addCommentNx(profile, user_comment, track, comment, profileGraph):
            
def addTrackCommenters(profile, track_commenters, profileGraph):
    for track, comment, track_commenter in track_commenters:
        addCommentNx(track_commenter, profile, track, comment, profileGraph):
            

# print graph functions

In [15]:
def printGraph(G):
    for profile, neighbor, key, weight in G.edges(data=True,keys=True):
        print("Profile: {}".format(userid2username(profile)))
        print("Neighbor: {}".format(userid2username(neighbor)))
        print(key)
        print(weight)
#     for profile in G.nodes():
#         if profile:
#             try:
#                 profile_name = id2username(profile)
#                 print("\t", "Printing: %s (%s)" % (profile_name, profile))
#                 # why does successors return an iterator instead of a list? getting fancy for no reason
#                 follows = [follow for follow in G.successors(profile)]
#                 followers = [follower for follower in G.predecessors(profile)]
#                 try:    
#                     print("\t", profile_name + " has " + str(len(follows)) + " followings")
#                     print("\t", profile_name + " follows " + ", ".join(map(lambda x: id2username(x), follows)))
#                 except TypeError: 
#                     print("No follows here!")
#                 try:    
#                     print("\t", profile_name + " has " + str(len(followers)) + " followers")
#                     print("\t", profile_name + " is followed by " + ", ".join(map(lambda x: id2username(x), followers)))
#                 except TypeError:
#                     print("No followers here!")
#                     print("-"*40)
#             except UnicodeError:
#                 print("Artist's username not found" )   

# 1b. Testing Functions

In [16]:
# list of profiles to query
profiles_to_query = [search.id]
profiles_to_query

[13377746]

In [17]:
depth = 1
i = 0

# list of profiles we could not query
unavailable_profiles = []

Only uncomment the code in the loop if you are willing to wait a while to run the cell below:

In [1]:
for t in range(depth):
    print("Iteration " + str(t))
    profiles_to_query = list(set(profiles_to_query))
    
    for profile in profiles_to_query:
        username = userid2username(profile)
        if username:
            print("\t", "Enqueueing: %s (%s)" % (username, profile))
            addProfileNx(profile,profile_graph)

#             encountered_follows = getFollows(profile)
#             encountered_follow_profiles = [follow.id for follow in encountered_follows]
#             print("encountered follows: " + ", ".join([userid2username(profile) for profile in encountered_follow_profiles]))
#             print('\n')
#             addFollowsNx(profile, encountered_follow_profiles, profile_graph)

#             encountered_user_favorites = getUserFavorites(profile)
#             encountered_user_favorite_tracks_profiles = [(track.title, getUserid(track) if isinstance(getUserid(track), int) else track.user_id) for track in encountered_user_favorites]
#             print("encountered user favorite profiles: " + ", ".join([userid2username(profile) for track_title, profile in encountered_user_favorite_tracks_profiles]))
#             print('\n')
#             addUserFavoritesNx(profile, encountered_user_favorite_tracks_profiles, profile_graph)

#             encountered_user_comments = getUserComments(profile)
#             encountered_user_comment_profiles = [trackid2userid(comment.track_id) for comment in encountered_user_comments]
#             print("encountered user comment profiles: " + ", ".join([userid2username(profile) for profile in encountered_user_comment_profiles]))
#             addUserCommentsNx(profile, encountered_user_comment_profiles, profile_graph)
#             print('\n')

#             encountered_followers = getFollowers(profile)
#             encountered_follower_profiles = [follower.id for follower in encountered_followers]
#             print("encountered followers: " + ", ".join([userid2username(profile) for profile in encountered_follower_profiles]))
#             print('\n')
#             addFollowersNx(profile, encountered_follower_profiles, profile_graph)

            encountered_tracks = getTracks(profile)

            encountered_track_comments = []
            encountered_comment_tracks = []
            encountered_track_favoriters = []
            encountered_favoriter_tracks = []

            for track in encountered_tracks:
                
                # grab track metadata
                track_title = track.title
                track_id = track.id
                
                #track number of comments / favoriters before adding new ones
                encountered_track_comments_len_before = len(encountered_track_comments)
                encountered_track_favoriters_len_before = len(encountered_track_favoriters)
                
                
                print("getting comments / favoriters from user track, {}".format(track_title))
                
                
                encountered_track_comments += getTrackComments(track_id)
                encountered_comment_tracks += (len(encountered_track_comments) - encountered_track_comments_len_before)*[track_title]
                encountered_track_favoriters += getTrackFavoriters(track_id)
                encountered_favoriter_tracks += (len(encountered_track_favoriters) - encountered_track_favoriters_len_before)*[track_title]
            
            encountered_track_comments_len = len(encountered_track_comments)
            encountered_comment_tracks_len = len(encountered_comment_tracks)
            encountered_track_favoriters_len = len(encountered_track_favoriters)
            encountered_favoriter_tracks_len = len(encountered_favoriter_tracks)
            
            print(encountered_track_comments_len==encountered_comment_tracks_len)
            print(encountered_track_favoriters_len==encountered_favoriter_tracks_len)
            
            print("encountered {} track comments and {} track favoriters, generating {} comment tracks and {} favoriter tracks".format(encountered_track_comments_len, encountered_track_favoriters_len, encountered_comment_tracks_len, encountered_favoriter_tracks_len))
            
            encountered_track_comments = list(zip(encountered_comment_tracks, encountered_track_comments))
            encountered_track_favoriters = list(zip(encountered_favoriter_tracks, encountered_track_favoriters))
            

#             print(encountered_track_comments[0].__dict__)
#             encountered_track_commenters = [(comment.user['id'], comment.user['username']) for comment in encountered_track_comments]
#             encountered_track_commenter_ids = [encountered_track_commenter[0] for encountered_track_commenter in encountered_track_commenters]
#             encountered_track_commenter_usernames = [encountered_track_commenter[1] for encountered_track_commenter in encountered_track_commenters]
#             print("encountered track comments: " + ", ".join(encountered_track_commenter_usernames))
#             print('\n')
#             addTrackCommenters(profile, encountered_track_commenter_ids, profile_graph)
            
            # need to update code below to initially store results in a list, as above
            # grab all people who have favorited a track of the target artist
            print(encountered_track_favoriters)
            print(encountered_track_favoriters[0])
            print(encountered_track_favoriters[0][1])#.__dict__)
            encountered_track_favoriter_ids = [(track_title, encountered_track_favoriter.id) for track_title, encountered_track_favoriter in encountered_track_favoriters]
            #print(encountered_track_favoriter_ids)
            encountered_track_favoriter_usernames = [userid2username(encountered_track_favoriter_id) for track_title, encountered_track_favoriter_id in encountered_track_favoriter_ids]
            print(encountered_track_favoriter_usernames)
            print("encountered track favoriters: " + ", ".join([encountered_track_favoriter_username if isinstance(encountered_track_favoriter_username, str) else " ".join(encountered_track_favoriter_username) for encountered_track_favoriter_username in encountered_track_favoriter_usernames]))
            addTrackFavoriters(profile, encountered_track_favoriter_ids, profile_graph)
            
        else:
            print("\t", "Artist ID %s is not query-able" % profile)
            unavailable_profiles.append(profile)            

NameError: name 'depth' is not defined

In [2]:
printGraph(profile_graph)

NameError: name 'printGraph' is not defined

# 2. metadata functions

In [75]:
def getUserInfo(profile):
    info = ['id','permalink', 'username', 'avatar_url', 'country', 'city', 
            'website', 'track_count', 'followers_count', 'followings_count']
    return {i: getAttr(profile, i) for i in info}

def getTrackInfo(track):
    info = ['id', 'user_id', 'created_at', 'streamabale', 'downloadable',
            'playback_count', 'download_count', 'favoritings_count','comment_count']
    return {i: getAttr(track, i) for i in info}

def getUserFavInfo(fav):
    info = ['id', 'user_id','last_modified']
    return {i: getAttr(fav, i) for i in info}

def getTrackFavInfo(fav):
    info = ['id', 'username', 'last_modified']
    return {i: getAttr(fav, i) for i in info}

def getUserCommInfo(comm):
    info = ['id', 'user_id', 'track_id', 'timestamp']
    return {i: getAttr(comm, i) for i in info}

def getTrackCommInfo(comm):
    info = ['id', 'user_id', 'track_id','timestamp']
    return {i: getAttr(comm, i) for i in info}

# 3. dump graph

In [77]:
#nx.write_graphml(profile_graph, 'profile.graphml')

In [78]:
print("Done.")

Done.
